In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
#config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE
import os
from os.path import exists
import shutil

from vtk_to_tiff_py import save_imagej_tiff, save_imagej_tiff4d, tableauZCYX, liste_temps


def chargement_modele(nom_model: str, base_dir: str) :
    # chargement du modèle
    model = CARE(config=None, name=nom_model, basedir= base_dir)
    return model

def evaluation_data(test_data_dir:str, test_labels_dir: str, date:str, t:int, ch) :
    """ charge deux images du jeu de test : une donnée et une étiquette au pas de temps t."""
    
    datafile = test_data_dir + "/" + date + "_t" + str(t).zfill(3) + + "_" + str(ch).zfill +'.tif'
    labelfile = test_labels_dir + "/" + date + "_t" + str(t).zfill(3) + + "_" + str(ch).zfill +'.tif'
    
    x, y = imread(datafile), imread(labelfile)
    return x, y

def rmse(im1, im2) :
    return np.sqrt(np.mean((im1 - im2)**2 ))

def erreur_test(nom_modele, test_data_dir, test_labels_dir, date, ch):
    """ donne la rmse et la différence (absolue) entre les images génerées et les données de test
    pour tous les pas de temps contenus dans data_dir """
    
    liste_t = liste_temps(date, test_data_dir, ch)
    erreurs = np.zeros(len(liste_t))
    diff_temps = np.zeros(len(liste_t))
    
    modele = chargement_modele(nom_modele, base_dir = "C:/Users/bioAMD/Desktop/Nathan" )
    
    for i in range(len(liste_t)) :
        
        x,y = evaluation_data(test_data_dir, test_labels_dir, date, liste_t[i], ch)
        axes = "ZCYX"
    
        # prédictions du modèle
        restored = modele.predict(x, axes)
        
        erreurs[i] = rmse(y, restored)
        diff_temps[i] = rmse(y, x)
        
    return erreurs, diff_temps

# conversion des prédictions en TIFF
from vtk_to_tiff_py import save_imagej_tiff

def save_predictions(dir_path_samples, output_path, model_path, date, scale, tstop, tstart, tstep, 
                     ch, diff=False, dir_path_labels=""):
    
    # chargement du modèle préentraîné
    modele = chargement_modele(model_path, base_dir)
    
    # si un dossier n'existe pas, on le crée pour sauvegarder les images prédites
    newpath = output_path + "/pred_" + model_path[len(base_dir)+1:]
    if not os.path.exists(newpath):
        os.makedirs(newpath, mode = 0o777)
    
    # on s'assure que les pas de temps sélectionnées sont bien dans le répertoire
    l_temps = liste_temps(date, dir_path_samples, ch)
    set_temps = set()
    for t in l_temps :
        set_temps.add(t)
            
    print('fichier de sauvegarde:', newpath)
    for t in range(tstart, tstop, tstep) :
        
        # si le pas de temps n'est pas présent, on passe à la suite
        if t not in set_temps :
            print('fichier t =', t, "absent")
            continue 
            
        if diff :
            x,y = evaluation_data(dir_path_samples, dir_path_labels,date, t, ch)
        else :
            x = imread(dir_path_samples + "/" + date + "_t" + str(t).zfill(3)
                       + "_ch" + str(ch).zfill(2) +'.tif')
            
        axes = "ZCYX"
        
        # on enlève le canal avec la moins bonne prédiction (si diff est vrai), sinon on prend un canal au hasard
        restored = modele.predict(x, axes)
        
        if diff :
            if np.mean(np.abs(restored[:,0,:,:]-y[:,0,:,:])) < np.mean(np.abs(restored[:,1,:,:]-y[:,1,:,:])):
                restored = restored[:,0,:,:]
                y = y[:,0,:,:]
            else :
                restored = restored[:,1,:,:]
                y = y[:,1,:,:]
        else :
            restored = restored[:,np.random.randint(low=0, high=2),:,:]
        
        # sauvegarde 
        save_file = newpath + "/" + date + "_t" + str(t).zfill(3) + "_ch" + str(ch).zfill(2) +'.tif'
        if exists(save_file) :
            os.remove(save_file)
        save_imagej_tiff(save_file, restored, scale, units="um", com=None )
        
        if diff :
            diff_file = newpath + "/" + date + "_t" + str(t).zfill(3) + "_ch" + str(ch).zfill(2) +'.tif'
        
            #on supprime si le fichier existe deja
            if exists(diff_file) :
                os.remove(diff_file)
        
            save_imagej_tiff(diff_file, np.abs(restored-y), scale, units="um", com=None )
        
        print("fichier t =",t, "sauvegardé")
        
    return None
def move_files(input_dir, output_dir, date, t:int, all_files=False) :
    """ déplace le(s) fichier(s) correspondant au pas de temps t, ou bien tout un répertoire """
    
    files = os.listdir(input_dir)
    
    assert t <= 999
    if not all_files :
        for filename in files :
            if str(t).zfill(3) == filename[len(date) + 2: len(date)+5] :
                shutil.move(input_dir+"/"+filename, output_dir+"/"+filename)
    else :
        for filename in files :
            shutil.move(input_dir+"/"+filename, output_dir+"/"+filename)
            
    return None

num_modele = 5
base_dir = "C:/Users/bioAMD/Desktop/Nathan"
nom_model = base_dir + "/modele_interpolation" + str(num_modele)
date = "070418a"
scale = tuple(np.array([1.37,1.37,1.37]).astype(np.float32))
ch = 8

data = base_dir+"/Interpolation/dataUniform"
test_data_dir = data+ "/test/test_samples" 
test_labels_dir = data+"/test/test_labels"

stock_samples_path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/test/test_samples"
stock_labels_path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/test/test_labels"

#output_path = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/saved_predictions"

In [ ]:
# erreurs, diff_temps = erreur_test(nom_model, test_data_dir, test_labels_dir, date, ch)
# liste_t = liste_temps(date, test_labels_dir, ch)
# plt.plot(liste_t, erreurs, 'x--', color='blue', label='reconstruction')
# plt.plot(liste_t, diff_temps, 'x--', color='red', label="Pas de temps adjacents" )
# plt.xlabel('pas de temps')
# plt.ylabel('RMSE')
# plt.grid()
# plt.legend()
# plt.show()

history_path = base_dir + "/Interpolation/trainHistoryDict" +str(num_modele)+".txt"
hist = np.genfromtxt(history_path, skip_header = 1)

epoch = hist[:,0]
loss = hist[:,1]
mse = hist[:,2]
mae = hist[:,3]
val_loss = hist[:,4]
val_mse = hist[:,5]
val_mae = hist[:,-1]

plt.plot(epoch, mae, label='mae')
plt.plot(epoch, val_mae, label='val_mae')
plt.grid()
plt.legend()
plt.show()

plt.plot(epoch, mse, label='mse')
plt.plot(epoch, val_mse, label='val_mse')
plt.grid()
plt.legend()
plt.show()

plt.plot(epoch, loss, label="loss")
plt.plot(epoch, val_loss, label='val_loss')
plt.grid()
plt.legend()
plt.show()

In [ ]:
##changement de canal ch
#dir_pred = "C:/Users/bioAMD/Desktop/Nathan/TIFFdata070418/stock_Demistep/pas_temps"
#for file in os.listdir(dir_pred) :
#    if file[-12:-4] == "ch00diff" :
#        newname = file[:-12] + "ch06.tif"
#        if os.path.exists(newname) :
#            os.remove(newname)
#        os.rename(dir_pred +"/" + file, dir_pred + "/" + newname)
#    elif file[-8:-4] == "ch00" :
#        newname = file[:-8] + "ch05.tif"
#        if os.path.exists(newname) :
#            os.remove(newname)
#        os.rename(dir_pred +"/" + file, dir_pred + "/" + newname)

In [ ]:
# comparaison modèles 
model5 = base_dir + "/modele_interpolation" + str(5)
model4 = base_dir + "/modele_interpolation" + str(4)

erreurs4, diff_temps4 = erreur_test(model4, test_data_dir, test_labels_dir, date, ch)
erreurs5, diff_temps5 = erreur_test(model5, test_data_dir, test_labels_dir, date, ch)

epoch = np.arange(1, len(erreurs2)+1, step=1)

plt.plot(epoch, erreurs2,  "x--",label="Modèle 5 (199 images)")
plt.plot(epoch, erreurs4,  "x--",label='Modèle 4 (158 images)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
dir_samples_Demistep = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/"
output_predDemistep = "C:/Users/bioAMD/Desktop/Nathan/TIFFdata070418/stock_Demistep"
ch = 8

l_temps = liste_temps(date, output_predDemistep)
filen= os.listdir(output_predDemistep)
for files in filen :
    if files[:len(date)+2] != date+"_t" or files[-8:] != "_ch"+str(ch).zfill(2)+".tif" :
        filen.remove(files)
print(filen[-1])

for t in range(len(l_temps)) :
    if l_temps[t] % 2 == 1 :
        shutil.move(output_predDemistep + "/" + filen[t], dir_samples_Demistep+ "/" + filen[t]  )


save_predictions(dir_samples_Demistep, output_predDemistep, nom_model, date, scale, tstart=239, tstop=1092, tstep=1)

In [ ]:
#model = chargement_modele(nom_model, base_dir)

date= "070418a"
scaling = (1.34, 1.34, 1.34)
dir_input = "C:/Users/bioAMD/Desktop/Nathan/TIFFdata070418/frames_perdues"
dir_output = "C:/Users/bioAMD/Desktop/Nathan/TIFFdata070418/frames_perdues/pred_perdues"

In [ ]:
t = liste_temps(date, dir_output, ch=5)
# axes = "ZCYX"

# i = 0
# for file in os.listdir(dir_output) :
    
#     if file[:len(date)+2] != date+"_t" :
#         continue
    
#     # on sauvegarde les données à prédire
#     if t[i] % 2 == 1 :
        


In [ ]:
save_predictions(dir_input, dir_output,nom_model, date, scale, tstop=1087, tstart=241,tstep=2, ch=5)

In [ ]:
tif_noyaux = "C:/Users/bioAMD/Desktop/Nathan/TIFFdata070418/stock_mix/tif_noyaux"
tif_membranes = "C:/Users/bioAMD/Desktop/Nathan/TIFFdata070418/stock_mix/tif_membranes"

def range_fichiers(date, dir_input ,dir_train) :
    """ renomme les fichiers et les ranges dans les jeux d'entrainement et de test
    samples: date + _t 123 456 _ch00.tif 
    labels : date + _t 123 456 _ch00.tif """

    for filename in os.listdir(dir_input) :
        
        if len(filename) < 21 or filename[:len(date)+2] != date+"_t" :
            continue
    
        newname = filename
    
        if filename[-6:] == "4D.tif" :
            newname = filename[:-6] + ".tif"
        
        # données d'entrée du jeu d'entrainement
        if len(filename) == ( len(date) + len("_t") + 3 + len("_ch00.tif") ) + 3 :
            
            nbre = ( int(filename[len(date) + 2 : len(date) + 5]) + int(filename[len(date)+5: len(date)+8]) ) // 2 
            newname = newname[:len(date)+2] + str(nbre).zfill(3) + newname[len(date)+8:]
                        
            if os.path.exists(dir_train + "/train_samples/" + newname) :
                os.remove(dir_train + "/train_samples/" + newname)
                
            shutil.move(dir_input+"/"+filename, dir_train+"/train_samples/"+filename)
            os.rename(dir_train+"/train_samples/"+filename, dir_train+"/train_samples/"+newname)
        
        # étiquettes du jeu d'entrainement
        else :
            if os.path.exists(dir_train + "/train_labels/" + newname) :
                os.remove(dir_train + "/train_labels/" + newname)
                
            shutil.move(dir_input+"/"+filename, dir_train+"/train_labels/"+filename)
            os.rename(dir_train+"/train_samples/"+filename, dir_train+"/train_labels/"+newname)
            
    return None

In [ ]:
ch = 8
date = "070418a"
modele = chargement_modele(nom_model, base_dir)
axes = "ZCYX"
scale = tuple(np.float32([1.34]*3))

# on fusionne les images des noyaux avec celles des membranes (le réseau prend 4 canaux en entrée)
path_noyaux = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_noyaux"
path_membranes = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_membranes"

liste_noyaux = os.listdir(path_noyaux)
liste_membranes = os.listdir(path_membranes)

path_output = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_all"
i = 0
for im_noyau, im_membrane in zip(liste_noyaux, liste_membranes) :
    
    if i < 508 :
        i += 1
        continue 
    # compteurs 
    snoy, smem = 0,0
    while im_noyau[len(date)+2+snoy] != "_" :
        snoy += 1
    while im_membrane[len(date)+2+smem] != "_" :
        smem += 1
    if im_noyau[len(date)+2:len(date)+2+snoy] != im_membrane[len(date)+2:len(date)+2+smem] :
        print("erreur: t_noyaux != t_membranes")
        continue 
        
    print(im_noyau, "/", im_membrane)
    
    t = im_noyau[len(date)+2:len(date)+2+snoy]
    
    # données pour la prédiction des noyau et membrane
    xnoy = imread(path_noyaux+"/"+im_noyau)
    xmem = imread(path_membranes+"/"+im_membrane)
    
    # donnée en sortie
    X = np.zeros(shape= xnoy[:,0,0,0].shape +tuple([4]) + xnoy[0,0,:,:].shape)

    # 2 premiers canaux : noyaux. Deux derniers : membranes
    X[:,0:2,:,:] = xnoy
    X[:,2:4,:,:] = xmem
    
    # on passe les images dans le réseau
    Y_pred = modele.predict(X, axes, n_tiles=(1,1,2,2))
    print(t+" :",Y_pred.shape)
    
    
    # fichier de suavegarde
    newfile = path_output + "/" + date+"_t"+t+"_ch" + str(ch).zfill(2) + ".tif"
    print("nom save", newfile)
    
    save_imagej_tiff4d(newfile, Y_pred, scale, units= "um")
    i += 1

In [ ]:
help(save_imagej_tiff)

In [ ]:
input_vtk = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_all/070418a"
output_pred = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/saved_predictions/pred_modele_interpolation5"
output_data = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/data"

def mae(a1,a2) :
    return np.mean(np.abs(a1 - a2))



liste_temps = np.arange(0,501,1)
from vtk_to_tiff_py import vtkToArray, save_imagej_tiff
files = os.listdir(input_vtk)
for i, file in enumerate(files) :
    if file[-6:] !="vtk.gz" :
        continue
    print(file)
    s = 0
    while file[9+s] != "_" :
        s += 1
    t = int(file[9:9+s])
    array, scale = vtkToArray(input_vtk + "/" + file)
    
    if t %2 == 1 :
        save_imagej_tiff(output_pred+"/"+file, array, scale, units='um')
    else :
        save_imagej_tiff(output_data+"/"+file, array, scale, units="um" )
    

In [5]:
save_dir = "C:/Users/bioAMD/Desktop/Nathan/Interpolation/dataMix/dataDemistep/demistep_all"

from patches_emb_py import sort_timesteps

help(chargement_modele)
def mae(a1, a2) :
    return np.mean(np.abs(a1-a2))

Help on function chargement_modele in module __main__:

chargement_modele(nom_model: str, base_dir: str)



In [21]:
num_modele = 5
base_dir = "C:/Users/bioAMD/Desktop/Nathan"
nom_model = base_dir + "/modele_interpolation" + str(num_modele)

files = sort_timesteps("070418a", save_dir)
files_pred= [ (files[i], files[i+2]) for i in range(0,len(files)-2,2) ]
files_label = [ files[i] for i in range(len(files)) if i%2 == 1 ]

errors = np.zeros(len(files_label))
diff = np.zeros(len(files_label))

modele = chargement_modele(nom_model, base_dir)
scale = np.float32([1.367,1.367,1.37])


for i in range(len(files_pred)):

    f1, f2 = files_pred[i]
    x1, x2 = imread(save_dir+"/"+f1)[:,::2,...], imread(save_dir+"/"+f2)[:,1::2,...]
    X = np.zeros(shape= tuple([len(x1)]) + tuple([4]) + x1.shape[2:])
    X[:,::2,:,:] = x1
    X[:,1::2,:,:] = x2
     
    # donnée pour les prédictions
    X = X.astype(np.uint16)
    axes = "ZCYX"
    
    # étiquette 
    y = imread(save_dir+"/"+files_label[i])[:,0,:,:].astype(np.uint16)
   
    # prédictions du modèle 
    restored = modele.predict(X, axes, n_tiles=(1,1,2,2))[:,0,:,:]
    print(restored.shape, np.min(X[:,0:2,:,:]), np.max(X[:,0:2,:,:]), np.min(X[:,2:,:,:]), np.max(X[:,2:,:,:]))
    save_imagej_tiff("C:/Users/bioAMD/Desktop/Nathan/Interpolation/saved_predictions/pred_modele_interpolation5/"+files[i], restored, scale, "um")
    a = mae(restored, y)
    b = mae(X[:,0,:,:],y)
    c = 
    
    with open(base_dir+"/Interpolation/dataMix/predictions5.txt", 'a') as f :
        f.write(str(i) + " ")
        f.write(str(mae(restored, y)) + " ")
        f.write(str(mae(X[:,0,:,:],y)) + " ")
        f.write(str(mae(X[:,2,:,:], y)) + "\n")
        

Loading network weights from 'weights_best.h5'.


100%|██████████| 4/4 [00:22<00:00,  5.66s/it]


(104, 512, 512) 0 255 0 255


KeyboardInterrupt: 